In [33]:
import numpy as np
import scipy as sp
import scipy.stats
import sklearn as sk
import xgboost as xgb
from sklearn import tree
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

Primeiramente, vamos ler o dataset de treino e armazenar suas informações em memória. Guardamos também uma lista de identificadores de colunas, que relacionam o index de cada coluna no dataset com o index original de cada coluna. Agora no início essa relação é de completa igualdade.

Guardamos também o índice da coluna referente ao target de cada registro.

In [34]:
dataset = []
target_column = -1
with open("datasets/train_file.csv", 'r') as f:
    header = f.readline()
    header = header.split(',')
    for line in f:
        data = line[:-1].split(',')
        # Usamos float para tratar possíveis casos de números fracionários em meio a registros inteiros
        data = list(map(float, data))
        dataset.append(data)
dataset = np.array(dataset)
columnsId = list(range((dataset.shape)[1]))
target_column = len(columnsId)-1

Observamos em uma análise externa a este caderno que algumas colunas possuem todos os seus registros iguais. Portanto, queremos que essas colunas não façam parte do dataset final de treino, pois não acrescentam em nada aos resultados e deixam o processamento mais lento por aumentar o tamanho da matriz.

Para isso, marcamos todas as colunas que possuem desvio padrão dos seus registros igual a zero, e deletamos elas do dataset.

In [35]:
columnsToDelete = []
eps = 1e-10
for i in range((dataset.shape)[1]):
    column = dataset[:, i]
    stderr = np.std(column)
    if stderr < eps:
        columnsToDelete.append(i)

In [36]:
dataset = np.delete(dataset, columnsToDelete, 1)
# Deletamos algumas colunas, precisamos deletar os índiced dessas colunas em nossa relação de índices
for col in columnsToDelete[::-1]:
    del(columnsId[col])

Para explorar ao máximo a informação que foi dada no dataset, fizemos uma outra análise externa a este notebook que apontou para a necessidade de aplicarmos um hashing trick no nosso dataset, a fim de eliminar colunas cujos valores numéricos não representem relação de ordem de grandeza (1 não necessariamente é menor do que 2, se considerarmos o que aquela coluna significa), e substitui-las por colunas que formem um espaço vetorial das possíveis entradas da coluna original. Por exemplo:

Uma coluna possui somente valores 0, 1, 2, 5, 6, 8 e 9. Podemos dividir essa coluna em outras 7 colunas binárias, onde a presença de um 1 em uma coluna implica que o valor original daquele registro era igual ao valor associado à sua nova coluna. Assim:

$$X - 0,1,2,5,6,8,9$$
$$---------$$
$$0 - 1,0,0,0,0,0,0$$
$$0 - 1,0,0,0,0,0,0$$
$$1 - 0,1,0,0,0,0,0$$
$$1 - 0,1,0,0,0,0,0$$
$$2 - 0,0,1,0,0,0,0$$
$$6 - 0,0,0,0,1,0,0$$
$$5 - 0,0,0,1,0,0,0$$
$$8 - 0,0,0,0,0,1,0$$
$$9 - 0,0,0,0,0,0,1$$
$$0 - 1,0,0,0,0,0,0$$
$$1 - 0,1,0,0,0,0,0$$

Para tanto, contamos e armazenamos quantos valores únicos uma coluna possui em um dicionário. Se tivermos 7 ou menos registros únicos, aplicamos nossa técnica de hashing trick. Armazenamos quais colunas iremos "expandir" em outras.

In [37]:
columnsToExpand = []
for i in range((dataset.shape)[1]-1):
    counter = {}
    column = dataset[:, i]
    for element in column:
        if element in counter:
            counter[element] += 1
        else:
            counter[element] = 1
    if len(counter.keys()) <= 7:
        keys = list(counter.keys())
        if keys == list(map(int, keys)):
            columnsToExpand.append((i, counter))

In [38]:
newcolumns = []
for col, counter in columnsToExpand:
    column = dataset[:, col]
    for key in counter.keys():
        # Adicionamos nossas novas colunas na nossa relação de índices
        columnsId.append((col, key))
        newcolumns.append([1 if el == key else 0 for el in column])

Não podemos nos esquecer de apagar as colunas originais!

In [39]:
dataset = np.delete(dataset, [col[0] for col in columnsToExpand], 1)
for col, counter in columnsToExpand[::-1]:
    # Removemos as colunas originais da relação de índices
    del(columnsId[col])
# Precisamos atualizar quem é a coluna de target, agora que mexemos na nossa relação de índices
target_column = columnsId.index(target_column)

E agora, unir o dataset antigo com as novas colunas:

In [40]:
dimensions = dataset.shape
newdataset = np.empty((dimensions[0], dimensions[1]+len(newcolumns)))
newcolumns = np.transpose(np.array(newcolumns))
newdataset[:, :dimensions[1]] = dataset
newdataset[:, dimensions[1]:] = newcolumns

Vamos agora criar nosso modelo usando a biblioteca XGBoost (Extreme Gradient Boosting), usando nosso novo dataset com o hashing trick. Os parâmetros usados também são especificados abaixo:

In [140]:
param = {"objective":"binary:logistic", "booster":"gbtree", "eval_metric":"auc", "nthread":4, "scale_pos_weight":1,
         "eta":0.0202048, "max_depth":6, "subsample":0.6815, "colsample_bytree":0.701, "seed":111}
# Lembrando que precisamos remover a coluna de target do dataset, e usa-la somente para especificar os targets no train
dtrain = xgb.DMatrix(np.delete(newdataset, target_column, 1), label=list(newdataset[:, target_column].astype("uint8")))
bst = xgb.train(param, dtrain, 312)

Uma vez treinado, o modelo poderá ser aplicado no dataset de teste. Portanto, realizaremos o mesmo procedimento anterior para carregar o dataset de teste em memória:

In [141]:
test_dataset = []
with open("datasets/test_file.csv", 'r') as f:
    f.readline()
    for line in f:
        data = line[:-1].split(',')
        data = list(map(float, data))
        test_dataset.append(data)
test_dataset = np.array(test_dataset)

Devemos deletar todas as colunas que deletamos do dataset original, e também realizar o mesmo procedimento de hashing trick descrito anteriormente no nosso dataset de teste.

In [142]:
for col in columnsToDelete[::-1]:
    test_dataset = np.delete(test_dataset, col, 1)

In [143]:
newcolumns = []
for col, counter in columnsToExpand:
    column = test_dataset[:, col]
    for key in counter.keys():
        newcolumns.append([1 if el == key else 0 for el in column])

In [144]:
for col, counter in columnsToExpand[::-1]:
    test_dataset = np.delete(test_dataset, col, 1)

In [145]:
dimensions = test_dataset.shape
newtestdataset = np.empty((dimensions[0], dimensions[1]+len(newcolumns)))
newcolumns = np.transpose(np.array(newcolumns))
newtestdataset[:, :dimensions[1]] = test_dataset
newtestdataset[:, dimensions[1]:] = newcolumns

Finalmente, podemos usar nosso modelo para prever a probabilidade de cada registro ser da classe 1:

In [146]:
dtest = xgb.DMatrix(newtestdataset)
y_pred = bst.predict(dtest)

E por fim, armazenar o resultado em um arquivo de texto para ser carregado no site da competição:

In [147]:
with open("result.txt", 'w') as f:
    for pred in y_pred:
        f.write(str(pred)+'\n')
f.close()